In [ ]:
import os

import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R
from tqdm.auto import tqdm

from realtime_trees.utils.ros_proxys import Time
from realtime_trees.tree_manager import TreeManager

## Read data from g2o file

In [ ]:
from re import T

g2o_path = "/home/stud/freissml/storage/user/datasets/SaR_2024-07/euroc/results/mission.g2o"
payload_cloud_dir = os.path.join(os.path.dirname(g2o_path), "payload_clouds")
individual_cloud_dir = os.path.join(os.path.dirname(g2o_path), "individual_clouds")

# Load g2o file
g2o_data = {}
with open(g2o_path, "r") as f:
    for line in f.readlines():
        if not line.startswith("VERTEX_SE3:QUAT"):
            continue
        line = line.split()
        id = int(line[1])
        x, y, z, qx, qy, qz, qw = map(float, line[2:-2])
        secs, nsecs = map(int, line[-2:])
        vertex_time = Time(secs, nsecs)
        T = np.eye(4)
        T[:3, :3] = R.from_quat([qx, qy, qz, qw]).as_matrix()
        T[:3, 3] = [x, y, z]
        g2o_data[vertex_time] = {"id": id, "transform": T}

# load point cloud
payload_duration = None # 20
payload_num_indiv_clouds = 5
payloads = []
if not os.path.exists(payload_cloud_dir) or len(os.listdir(payload_cloud_dir)) == 0:
    print("Accumulating payloads from individual clouds")
    # accumulated payloads don't exist, merge individual payloads over timespans of payload_duration
    sorted_times = np.sort(list(g2o_data.keys()))
    individual_clouds = []
    for i_time, time in tqdm(enumerate(sorted_times)):
        file_path = os.path.join(individual_cloud_dir, f"cloud_{time.secs:0>10}_{time.nsecs:0>9}.pcd")
        if os.path.exists(file_path):
            cloud = o3d.io.read_point_cloud(file_path)
            individual_clouds.append({"cloud": cloud, "time_stamp": time})
        else:
            print(f"WARNING: Individual cloud at {file_path} not found!")
            
        if len(individual_clouds) > 1:
            if payload_duration is not None:
                if individual_clouds[-1]["time_stamp"] - individual_clouds[0]["time_stamp"] < payload_duration:
                    continue
            elif payload_num_indiv_clouds is not None:
                if len(individual_clouds) < payload_num_indiv_clouds:
                    continue
            else:
                raise ValueError("Either payload_duration or payload_num_indiv_clouds must be set")
            # merge clouds
            cloud = o3d.geometry.PointCloud()
            for cloud_data in individual_clouds:
                cloud_data["cloud"].transform(g2o_data[cloud_data["time_stamp"]]["transform"])
                cloud += cloud_data["cloud"]
            if not cloud.has_normals():
                cloud.estimate_normals()
            cloud = o3d.t.geometry.PointCloud.from_legacy(cloud)
            payloads.append({"cloud": cloud, "time_stamp": individual_clouds[0]["time_stamp"]})
            individual_clouds = []
else:
    # payloads are already accumulated, load them directly
    print("Loading payloads as stored in payload dir")
    for file in tqdm(os.listdir(payload_cloud_dir)):
        if ".pcd" not in file:
            continue
        secs, nsecs =  map(int, file.replace(".pcd", "").split("_")[1:])
        cloud_stamp = Time(secs, nsecs)
        cloud = o3d.io.read_point_cloud(os.path.join(payload_cloud_dir, file))
        if cloud_stamp not in g2o_data:
            print(f"WARNING: payload cloud at stamp {cloud_stamp} not in g2o file!")
            continue
        cloud.transform(g2o_data[cloud_stamp]["transform"])
        # compute normals if not already present
        if not cloud.has_normals():
            cloud.estimate_normals()
        cloud = o3d.t.geometry.PointCloud.from_legacy(cloud)
        payloads.append({"cloud": cloud, "time_stamp": cloud_stamp})

tree_manager = TreeManager()
with open(g2o_path, "r") as f:
    lla_ref = None
    lla_r2m = None
    for line in f.readlines():
        if line.startswith("#"):
            continue
        if "LLA" in line:
            if "REF" in line:
                lla_ref = np.array([float(x) for x in line.split("LLA_REF")[1].split()])
            if "TO_MAP" in line:
                lla_r2m = np.array([float(x) for x in line.split("TO_MAP")[1].split()])
tree_manager.register_gnss(lla_ref, lla_r2m)

## Cluster Individual Payloads and add to Tree Manager

In [ ]:
%load_ext autoreload
%autoreload 2
from realtime_trees.terrain import fit_terrain
from realtime_trees.clustering import cluster
from realtime_trees.utils.matrix_calc import efficient_inv
from realtime_trees.utils.meshing import meshgrid_to_mesh
from realtime_trees.tree_manager import TreeManager


for payload in tqdm(payloads):
    time_stamp = payload["time_stamp"]
    # Terrain simulation
    terrain = fit_terrain(
        cloud=payload["cloud"],
        sloop_smooth=False,
        cloth_cell_size=2.0
    )
    terrain_verts, terrain_tris = meshgrid_to_mesh(terrain)
    tree_manager.add_terrain(
        terrain_mesh_verts_map=terrain_verts,
        terrain_mesh_tris=terrain_tris,
        time_stamp=time_stamp,
        T_sensor2map=np.eye(4),
        cell_size=1.0 # same as in fit_terrain()
    )
    
    
    # mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(terrain_verts), o3d.utility.Vector3iVector(terrain_tris))
    # o3d.visualization.draw_geometries([payload["cloud"].to_legacy(), mesh])
    
    # Clustering
    clusters = cluster(
        cloud=payload["cloud"],
        terrain=terrain,
        # crop_bounds=[[0.5, 1.25], [1.5, 2.25], [2.5, 3.25], [3.5, 4.25], [4.5, 5.25]],
        crop_bounds=[[0.5, 1.5], [2., 3.], [4., 5.]],
        max_cluster_radius=5.0,
    )
    print(f"found {len(clusters)} clusters")
    
    # clouds = []
    # for c in clusters:
    #     clouds.append(c["cloud"].to_legacy())
    #     clouds[-1].paint_uniform_color(np.random.rand(3)) 
    # o3d.visualization.draw_geometries(clouds)
    
    # convert clusters into stamped sensor frame
    for i in range(len(clusters)):
        clusters[i].cloud.transform(efficient_inv(g2o_data[time_stamp]["transform"]))
        clusters[i].info.axis.transform = (
            efficient_inv(g2o_data[time_stamp]["transform"]) @ clusters[i].info.axis.transform
        )
        clusters[i].info.T_sensor2map = g2o_data[time_stamp]["transform"]
        clusters[i].info.time_stamp = time_stamp
    tree_manager.add_clusters(clusters, try_reconstruction=False) # perform reconstruction in the end

## Reconstruct the Trees

In [ ]:
for tree in tqdm(tree_manager.trees):
    tree.reconstruct(
        slice_heights=1.0,
        max_consecutive_fails=10,
        filter_radius=0.05,
        max_center_deviation=0.2,
        max_radius=0.5)
    tree_manager.analyze_tree(tree)
print(f"Successfully reconstructed {np.sum([tree.reconstructed for tree in tree_manager.trees])} trees")

In [ ]:
viz_objects = []
import random
for tree in random.sample(tree_manager.trees, len(tree_manager.trees)):
    pc = o3d.t.geometry.PointCloud(tree.points).to_legacy()
    pc.paint_uniform_color([np.random.uniform(0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1)])
    viz_objects.append(pc)
    
    if tree.reconstructed:
        verts, tris = tree.generate_mesh()
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(verts)
        mesh.triangles = o3d.utility.Vector3iVector(tris)
        mesh.compute_vertex_normals()  # Compute vertex normals for shading
        mesh.paint_uniform_color([0, 1, 0])  # Set mesh color to green and 50% transparent
        viz_objects.append(mesh)

o3d.visualization.draw_geometries(viz_objects)

## Export

In [ ]:
from posixpath import dirname

export_dir = os.path.join(dirname(dirname(dirname(g2o_path))), "realtime-trees")
os.makedirs(export_dir, exist_ok=True)
exp_name = dirname(export_dir).split("/")[-1]
tree_manager.write_report(export_dir)
tree_manager.save_as_zip(os.path.join(export_dir, f"tree_manager_{exp_name}"))

## Plot

In [ ]:
from matplotlib import pyplot as plt
from realtime_trees.utils.gnss import map2lla


recod_trees = [tree for tree in tree_manager.trees if tree.dbh is not None]


fig, ax = plt.subplots(1,1, figsize=(10,5))
tree_centers = []
ax.set_aspect("equal")
for tree in recod_trees:
    center = map2lla(tree.axis.transform[:3, 3], tree_manager.lla_ref, tree_manager.lla_r2m)
    tree_centers.append(center)
    dbh = tree.dbh
    ax.add_artist(plt.Circle([center[1], center[0]], 2e-5*dbh/2, fill=True, color="#10AA00", label="Hough"))
    ax.add_artist(plt.Circle([center[1], center[0]], 2e-5*dbh/2, fill=False, color="#000000", label="Hough"))
    ax.text(center[1]-1.5e-5, center[0]-dbh*2e-5-1.2e-5, f"{tree.id:>0}", fontsize=9)
tree_centers = np.array(tree_centers)
ax.set_xlim(tree_centers[:, 1].min()-5e-5, tree_centers[:, 1].max()+5e-5)
ax.set_ylim(tree_centers[:, 0].min()-5e-5, tree_centers[:, 0].max()+5e-5)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
_ = ax.set_title("Marteloscope of Tree Manager")